# Tutorial 1: Building ZenML Pipelines

This tutorial demonstrates how to build a complete ZenML pipeline.


In [1]:
import torch

from torch_geometric.data import Data
from zenml import pipeline, step

from pioneerml.models import GroupClassifier
from pioneerml.training import GraphDataModule, GraphLightningModule
from pioneerml.zenml.materializers import (
    GraphDataModuleMaterializer,
    PyGDataListMaterializer,
)
from pioneerml.evaluation.plots import plot_loss_curves
from pioneerml.zenml import load_step_output
from pioneerml.zenml import utils as zenml_utils

# Initialize ZenML for notebook use
# setup_zenml_for_notebook automatically finds the project root by searching
# upward for .zen or .zenml directories, ensuring we use the root configuration.
# use_in_memory=True creates a temporary in-memory SQLite store, perfect for
# tutorials where we don't need persistent artifact storage.
zenml_client = zenml_utils.setup_zenml_for_notebook(use_in_memory=True)
print(f"ZenML initialized with stack: {zenml_client.active_stack_model.name}")





Using ZenML repository root: /home/jack/python_projects/pioneerML
Ensure this is the top-level of your repo (.zen must live here).
Creating database tables


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 # use_in_memory=True creates a temporary in-memory SQLite store, perfect for                │
│    6 # tutorials where we don't need persistent artifact storage.                                │
│    7 zenml_client = zenml_utils.setup_zenml_for_notebook(use_in_memory=True)                     │
│ ❱  8 print(f"ZenML initialized with stack: {zenml_client.active_stack_model.name}")              │
│    9                                                                                             │
│   10                                                                                             │
│   11 import torch                                                                                │
│                                                                                                  │
│ /home/jack/virtual_environments/miniconda3/envs/pioneerml/lib/python3.11/site-packages/zenml/cli │
│ ent.py:1541 in active_stack_model                                                                │
│                                                                                                  │
│   1538 │   │   if not stack_id:                                                                  │
│   1539 │   │   │   # Initialize the zen store so the global config loads the active              │
│   1540 │   │   │   # stack                                                                       │
│ ❱ 1541 │   │   │   _ = GlobalConfiguration().zen_store                                           │
│   1542 │   │   │   if active_stack := GlobalConfiguration()._active_stack:                       │
│   1543 │   │   │   │   return active_stack                                                       │
│   1544                                                                                           │
│                                                                                                  │
│ /home/jack/virtual_environments/miniconda3/envs/pioneerml/lib/python3.11/site-packages/zenml/con │
│ fig/global_config.py:230 in __custom_getattribute__                                              │
│                                                                                                  │
│   227 │   │   Returns:                                                                           │
│   228 │   │   │   The attribute value.                                                           │
│   229 │   │   """                                                                                │
│ ❱ 230 │   │   value = super().__getattribute__(key)                                              │
│   231 │   │   if key.startswith("_") or key not in type(self).model_fields:                      │
│   232 │   │   │   return value                                                                   │
│   233                                                                                            │
│                                                                                                  │
│ /home/jack/virtual_environments/miniconda3/envs/pioneerml/lib/python3.11/site-packages/zenml/con │
│ fig/global_config.py:724 in zen_store                                                            │
│                                                                                                  │
│   721 │   │   │   The current zen store.                                                         │
│   722 │   │   """                                                                                │
│   723 │   │   if self._zen_store is None:                                                        │
│ ❱ 724 │   │   │   self._configure_store(self.store_configur

## Define the Training Pipeline

Here we build the pipeline steps directly in the notebook so you can see exactly how ZenML pipelines are composed. The cell below defines each step:

1. **`create_data`**: Generates synthetic graph data for a 3-class classification task
   - Uses `PyGDataListMaterializer` to serialize PyTorch Geometric data efficiently (avoids pickle warnings)
   
2. **`create_datamodule`**: Prepares the data for training
   - Splits data into train/validation sets (80/20 split)
   - Configures batch size and data loading workers
   
3. **`create_model`**: Instantiates the neural network architecture
   - Creates a `GroupClassifier` with appropriate hyperparameters
   
4. **`create_lightning_module`**: Wraps the model in PyTorch Lightning
   - Adds training logic, loss function, and optimizer configuration
   
5. **`train_model`**: Executes the training loop
   - Automatically detects available hardware (CPU/GPU)
   - Runs training for a specified number of epochs

**How ZenML wires steps together:** Steps communicate via return values. ZenML automatically matches step outputs to the next step's inputs based on function parameter names. Custom **materializers** control how artifacts are serialized to disk, keeping storage efficient and warnings quiet.



In [ ]:

def create_simple_synthetic_data(num_samples: int = 200) -> list[Data]:
    """Generate clustered graphs so the model can learn a clear signal.
    
    Each class has distinct feature patterns:
    - pi cluster: boost feature 0
    - mu cluster: boost feature 1  
    - e+ cluster: negative drift across features
    """
    class_offsets = torch.tensor(
        [
            [2.0, 0.0, 0.5, 0.0, 0.0],   # pi cluster: boost feature 0
            [0.0, 2.0, 0.0, 0.5, 0.0],   # mu cluster: boost feature 1
            [-2.0, -2.0, -0.5, 0.0, 0.5],  # e+ cluster: negative drift
        ]
    )
    data: list[Data] = []
    for _ in range(num_samples):
        num_nodes = torch.randint(6, 10, (1,)).item()
        label = torch.randint(0, 3, (1,)).item()

        # Clustered node features with light noise make class boundaries learnable
        x = torch.randn(num_nodes, 5) * 0.4 + class_offsets[label]

        # Random edges with low-variance attributes keep the task simple
        edge_index = torch.randint(0, num_nodes, (2, num_nodes * 3))
        edge_attr = torch.randn(edge_index.shape[1], 4) * 0.3

        # One-hot encoded labels for multilabel classification
        y = torch.zeros(3)
        y[label] = 1.0
        data.append(Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y))
    return data


@step(output_materializers=PyGDataListMaterializer, enable_cache=False)
def create_data() -> list[Data]:
    """Step 1: Generate synthetic graph data.
    
    The PyGDataListMaterializer ensures efficient serialization of PyTorch Geometric
    Data objects, avoiding pickle warnings.
    """
    return create_simple_synthetic_data()


@step(output_materializers=GraphDataModuleMaterializer, enable_cache=False)
def create_datamodule(data: list[Data]) -> GraphDataModule:
    """Step 2: Create data module with train/val split.
    
    - val_split=0.2: 80% train, 20% validation
    - batch_size=32: slightly larger for faster iteration
    - num_workers=0: keeps it portable for sandboxed runs
    """
    return GraphDataModule(dataset=data, val_split=0.2, batch_size=32, num_workers=0)


@step
def create_model(num_classes: int = 3) -> GroupClassifier:
    """Step 3: Instantiate the neural network architecture.
    
    Smaller hidden dim (64) and block count (1) to keep the tutorial quick on CPU.
    """
    return GroupClassifier(num_classes=num_classes, hidden=64, num_blocks=1)


@step
def create_lightning_module(model: GroupClassifier) -> GraphLightningModule:
    """Step 4: Wrap model in PyTorch Lightning module.
    
    Adds training logic, loss function (BCE for multilabel), and optimizer config.
    """
    return GraphLightningModule(model, task="classification", lr=1e-3)


@step
def train_model(lightning_module: GraphLightningModule, datamodule: GraphDataModule) -> GraphLightningModule:
    """Step 5: Execute training loop.
    
    Automatically detects available hardware (CPU/GPU) and runs for 3 epochs.
    Returns the trained module in eval mode.
    """
    import pytorch_lightning as pl
    from pioneerml.zenml.utils import detect_available_accelerator

    accelerator, devices = detect_available_accelerator()

    trainer = pl.Trainer(
        accelerator=accelerator,
        devices=devices,
        max_epochs=10,
        logger=False,
        enable_checkpointing=False,
        enable_progress_bar=False,
    )
    trainer.fit(lightning_module, datamodule=datamodule)
    return lightning_module.eval()


@pipeline
def basic_training_pipeline_demo():
    """Compose all steps into a complete training pipeline.
    
    ZenML automatically wires the outputs of one step to the inputs of the next
    based on parameter names. For example, `create_datamodule(data)` receives
    the output from `create_data()`.
    """
    data = create_data()
    datamodule = create_datamodule(data)
    model = create_model()
    lightning_module = create_lightning_module(model)
    trained_module = train_model(lightning_module, datamodule)
    return trained_module, datamodule



## Run the Pipeline

Execute the pipeline and load the artifacts. After the pipeline completes, we load the trained model and datamodule using `load_step_output`. These artifacts are stored by ZenML and can be reloaded anytime without re-running the pipeline.

**Why use `enable_cache=False`?** This ensures the pipeline runs fresh each time, which is useful for tutorials. In production, you'd typically enable caching to skip re-running unchanged steps.


In [ ]:
run = basic_training_pipeline_demo.with_options(enable_cache=False)()
print(f"Pipeline run status: {run.status}")

# Load artifacts from the run so we can inspect them locally
trained_module = load_step_output(run, "train_model")
datamodule = load_step_output(run, "create_datamodule")
model = getattr(trained_module, "model", None) if trained_module is not None else None

if trained_module is None or datamodule is None:
    raise RuntimeError("Could not load artifacts from the basic_training_pipeline run.")

# Move model to appropriate device and set up datamodule for evaluation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_module = trained_module.to(device).eval()
datamodule.setup(stage="fit")

print(f"Loaded artifacts from run {run.name} (device={device})")
plot_loss_curves(trained_module, title="Tutorial 1: Loss Curves", show=True)



## Inspect the Outputs

Check the dataset size, batch shape, and key model parameters. This helps validate that the data/materializers round-tripped correctly and that the model configuration matches expectations.



In [ ]:
device = next(trained_module.parameters()).device
train_loader = datamodule.train_dataloader()
first_batch = next(iter(train_loader))
train_size = len(datamodule.train_dataset) if datamodule.train_dataset is not None else 0
val_size = len(datamodule.val_dataset) if datamodule.val_dataset is not None else 0

print("Training summary:")
print(f"- Run: {run.name}")
print(f"- Device: {device}")
print(f"- Dataset size: {train_size + val_size} samples (train={train_size}, val={val_size})")
print(f"- Batch shape: x={tuple(first_batch.x.shape)}, edge_index={tuple(first_batch.edge_index.shape)}")

if model:
    print("\nModel configuration:")
    print(f"- Type: {type(model).__name__}")
    print(f"- Hidden dimension: {getattr(model, 'hidden', 'n/a')}")
    print(f"- Num classes: {getattr(model, 'num_classes', 'n/a')}")



## Evaluate on Validation Set

Compute a lightweight accuracy metric as a sanity check that training worked. Accuracy here is on a tiny synthetic validation set, so treat it as a smoke test rather than a real benchmark.

**How it's computed:**
- Apply `argmax` to model logits to get predicted class IDs
- Apply `argmax` to one-hot labels to get true class IDs  
- Compare predictions vs targets for exact match
- Average over the validation set

**Note on accuracy vs loss:** Accuracy measures whether the predicted class matches the true class (hard decision). Loss (BCE) measures how confident the model is - even with 100% accuracy, loss can be non-zero if logits aren't at the extremes. This is normal and expected.

**Interpretation:** >0.33 means the model is learning above random chance (3 classes).



In [ ]:
val_loader = datamodule.val_dataloader()
# Verify we're using the validation set
val_size = len(datamodule.val_dataset) if datamodule.val_dataset is not None else 0
if val_size == 0:
    print("Warning: Validation dataset is empty, falling back to train loader")
    val_loader = datamodule.train_dataloader()
else:
    print(f"Evaluating on validation set ({val_size} samples)")

correct = 0
total = 0

for batch in val_loader:
    batch = batch.to(device)
    with torch.no_grad():
        logits = trained_module(batch)
    
    # Convert one-hot labels to class indices
    # batch.y should be [num_graphs_in_batch, num_classes] when batched
    labels = batch.y
    
    # Handle edge case: if labels are 1D, reshape them
    if labels.dim() == 1:
        # If labels are flattened, reshape based on logits shape
        if logits.shape[-1] > 0 and labels.numel() % logits.shape[-1] == 0:
            labels = labels.view(-1, logits.shape[-1])
        else:
            # If labels are already class indices (1D), keep them as is
            preds = torch.argmax(logits, dim=1)
            correct += int((preds == labels).sum().item())
            total += int(labels.numel())
            continue
    
    # Convert one-hot labels [num_graphs, num_classes] to class indices [num_graphs]
    if labels.dim() > 1:
        labels = torch.argmax(labels, dim=1)
    
    # Get predicted class indices from logits [num_graphs, num_classes] -> [num_graphs]
    preds = torch.argmax(logits, dim=1)
    
    # Count correct predictions
    correct += int((preds == labels).sum().item())
    total += int(labels.numel())

accuracy = correct / total if total else 0.0
print(f"Validation accuracy (quick check): {accuracy:.3f} ({correct}/{total} correct)")
